### import 및 함수

In [7]:
import os, re, functions
import pandas as pd
from os.path import join
from datetime import datetime

In [15]:
# 파일읽기
path_read_dir = r"\\192.168.0.75\자산관리팀\프로젝트\2023 전산데이터 클렌징 및 업무개편 프로젝트\법조치\2509 회계파산"
path_read_fn = "250910 일단만료 사건조회_20250911_1238.xlsx"

df_ori = pd.read_excel(join(path_read_dir, path_read_fn), dtype=str).fillna("")

In [51]:
# 조회성공건
df = df_ori[df_ori.최신화결과==""].copy()

In [49]:
def extract_parties(text):
    원고, 피고 = [], []
    피승계인, 승계인, 승계송달발급일 = [], [], []

    corp_prefix = {"주식회사", "유한회사", "합자회사", "법인"}
    pattern = re.compile(r'(원고|채권자|채무자|피고)\d*\s+([^\s]+)')
    ignore_keywords = ("제3채무자", "대리인", "지배인")

    for line in text.split("\n"):

        # 피고 승계인 패턴 먼저 확인
        m = re.search(r'(피고|채무자)\d*\s+([^\s]+)의\s+승계인\s+([^\s]+)에게', line)
        if m:
            # m.group(2) = 피승계인, m.group(3) = 승계인
            피승계인.append(m.group(2))
            승계인.append(m.group(3))
            continue
        
        # 원고/채권자 승계인 라인에서 발급일만 추출
        m = re.search(r'(원고|채권자).*?승계인.*?(\d{4})[.](\d{2})[.](\d{2}).*?발급\s*$', line)
        if m:
            # 발급일을 YYYY-MM-DD 형식으로 저장
            발급일 = f"{m.group(2)}-{m.group(3)}-{m.group(4)}"
            승계송달발급일.append(발급일)
            continue
        
        # 원고, 피고 추출 시작
            # 탐색불요 라인
        if any(key in line for key in ignore_keywords):
            continue
        if "에게" not in line:
            continue
        
        for role, name in pattern.findall(line):
            parts = line.split()
            if name in corp_prefix:  # 법인 접두어 제거
                idx = parts.index(name)
                if idx + 1 < len(parts):
                    name = parts[idx + 1]

            # 괄호 및 뒷부분 제거
            name = re.sub(r"\(.*?\).*", "", name)

            # 조사 제거
            name = re.sub(r"(에게|에|의)$", "", name)

            name = name.strip()
            if not name:
                continue

            # 분류
            if role == "원고" or role == "채권자":
                원고.append(name)
            elif role == "피고" or role == "채무자":
                피고.append(name)
            # elif role == "채권자":
            #     채권자.append(name)
            # elif role == "채무자":
            #     채무자.append(name)

    uniq = lambda x: list(dict.fromkeys(x))
    
    if 승계송달발급일:  # 리스트가 비어있지 않은 경우
        최종승계송달발급일 = max(datetime.strptime(d, "%Y-%m-%d") for d in 승계송달발급일)
        최종승계송달발급일 = 최종승계송달발급일.strftime("%Y-%m-%d")
    else:
        최종승계송달발급일 = ''
    
    return {
    "원고": ",".join(uniq(원고)),
    "피고": ",".join(uniq(피고)),
    # "채권자": ",".join(uniq(채권자)),
    # "채무자": ",".join(uniq(채무자)),
    "피승계인": ",".join(uniq(피승계인)),
    "승계인": ",".join(uniq(승계인)),
    '최종승계송달발급일': 최종승계송달발급일
}


In [52]:
# 원고, 피고 열 없애기
df = df.drop(columns={"원고", "피고"})

df_parties = df["진행내용"].apply(extract_parties).apply(pd.Series)
df = pd.concat([df, df_parties], axis=1)

In [ ]:
# 대상자구분 작성하기

70      
71      
72      
73      
74      
      ..
331     
332     
333     
334     
335     
Name: 채무자명, Length: 266, dtype: object

In [ ]:
# 헬프승계여부 : 

In [ ]:
# 이것들은 나우리 전산 수정하기보다는 새로 만들 업무페이지에 업로드 하는 걸로 할까? 어차피 진행내용이 법조치 조회에서는 안 나오고 사건조회를 해야만 하니까.. 
# 그렇다면 진행내용 통해서 피고들 중 누구만 확정됐는지, 누구는 취하했는지 다 파악해야 함